## This Notebook demonstrates how to reduce the bias during "Pre-processing" & "In-processing" stage using AI 360 Fairness toolkit

### Pre-processing algorithm
A bias mitigation algorithm that is applied to training data.

### In-processing algorithm
A bias mitigation algorithm that is applied to a model during its training.

### Insert your credentials as credentials in the below cell
Click on dropdown from Pipeline_LabelEncoder-0.1.zip under Data tab and select 'Credentials'

In [1]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials = {
    
    
}


In [2]:
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [3]:
import os
os.getcwd()

'/home/wsuser/work'

In [4]:
cos.download_file(Bucket=credentials['BUCKET'],Key='Pipeline_LabelEncoder-0.1.zip',Filename='/home/wsuser/work/Pipeline_LabelEncoder-0.1.zip')

In [5]:
!ls

Pipeline_LabelEncoder-0.1.zip


In [6]:
!pip install Pipeline_LabelEncoder-0.1.zip
!pip install aif360
!pip install 'tensorflow>=1.13.1,< 2' --force-reinstall

Processing ./Pipeline_LabelEncoder-0.1.zip
  Created wheel for Pipeline-LabelEncoder: filename=Pipeline_LabelEncoder-0.1-py3-none-any.whl size=2060 sha256=7346c21ec588a2e781a6387ad7798665630aa065f85a0905a58ac80b016354b2
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/a1/1a/b1/66d8f1917ec5b09eb70adf911c60dec54820888fd7cb9941ad
Successfully built Pipeline-LabelEncoder
  Attempting uninstall: Pipeline-LabelEncoder
    Found existing installation: Pipeline-LabelEncoder 0.1
    Uninstalling Pipeline-LabelEncoder-0.1:
      Successfully uninstalled Pipeline-LabelEncoder-0.1
  Using cached tensorflow-1.15.5-cp37-cp37m-manylinux2010_x86_64.whl (110.5 MB)
  Using cached protobuf-3.14.0-cp37-cp37m-manylinux1_x86_64.whl (1.0 MB)
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
Processing /tmp/wsuser/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6/wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl
Processing /tmp/wsuser/.cache/pip/wheels/3f/e3/ec/8a8336

In [7]:
import tensorflow as tf
tf.__version__

'1.15.5'

In [8]:
%matplotlib inline
# Load all necessary packages
import pandas as pd
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

In [9]:


df = pd.read_csv(body)
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,Old,1
1,1,85,66,29,0,26.6,0.351,Young,0
2,8,183,64,0,0,23.3,0.672,Young,1
3,1,89,66,23,94,28.1,0.167,Young,0
4,0,137,40,35,168,43.1,2.288,Young,1


In [10]:
df.describe(include = 'all')

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768,768.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Young,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,679,NaN
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,NaN,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,NaN,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,NaN,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,NaN,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,NaN,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,NaN,1.000000


In [11]:
privileged_groups = [{'Age': 1}]
unprivileged_groups = [{'Age': 0}]
favorable_label = 0 
unfavorable_label = 1

In [12]:
from sklearn import preprocessing
categorical_column = ['Age']

data_encoded = df.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical_column:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)
        

data_encoded.head()

Feature Age
mapping {'Old': 0, 'Young': 1}


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,0,1
1,1,85,66,29,0,26.6,0.351,1,0
2,8,183,64,0,0,23.3,0.672,1,1
3,1,89,66,23,94,28.1,0.167,1,0
4,0,137,40,35,168,43.1,2.288,1,1


In [13]:
from Pipeline_LabelEncoder.sklearn_label_encoder import PipelineLabelEncoder
preprocessed_data = PipelineLabelEncoder(columns = ['Age']).fit_transform(data_encoded)
print('-------------------------')
#print('validation data encoding')
#validation_enc_data = PipelineLabelEncoder(columns = ['Gender','Married', 'Fraud_risk']).transform(validation_input_data)

Inside fit transform
Feature Age
mapping {0: 0, 1: 1}
-------------------------


In [14]:
#Create binary label dataset that can be used by bias mitigation algorithms
diabetes_dataset = BinaryLabelDataset(favorable_label=favorable_label,
                                unfavorable_label=unfavorable_label,
                                df=preprocessed_data,
                                label_names=['Outcome'],
                                protected_attribute_names=['Age'],
                                unprivileged_protected_attributes=unprivileged_groups)

In [15]:
display(Markdown("#### Training Data Details"))
print("shape of the training dataset", diabetes_dataset.features.shape)
print("Training data favorable label", diabetes_dataset.favorable_label)
print("Training data unfavorable label", diabetes_dataset.unfavorable_label)
print("Training data protected attribute", diabetes_dataset.protected_attribute_names)
print("Training data privileged protected attribute (1:Young and 0:Old)", 
      diabetes_dataset.privileged_protected_attributes)
print("Training data unprivileged protected attribute (1:Young and 0:Old)",
      diabetes_dataset.unprivileged_protected_attributes)

#### Training Data Details

shape of the training dataset (768, 8)
Training data favorable label 0.0
Training data unfavorable label 1.0
Training data protected attribute ['Age']
Training data privileged protected attribute (1:Young and 0:Old) [array([1.])]
Training data unprivileged protected attribute (1:Young and 0:Old) [array([0.])]


In [16]:
diabetes_dataset_train, diabetes_dataset_test = diabetes_dataset.split([0.6], shuffle=True)

In [17]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(diabetes_dataset_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(diabetes_dataset_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.099364
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.241212


In [18]:
min_max_scaler = MaxAbsScaler()
diabetes_dataset_train.features = min_max_scaler.fit_transform(diabetes_dataset_train.features)
diabetes_dataset_test.features = min_max_scaler.transform(diabetes_dataset_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(diabetes_dataset_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(diabetes_dataset_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.099364
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.241212


### Build plan classifier without debiasing

In [19]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
#sess = tf.compat.v1.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [20]:
plain_model.fit(diabetes_dataset_train)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





epoch 0; iter: 0; batch classifier loss: 0.733848
epoch 1; iter: 0; batch classifier loss: 0.705918
epoch 2; iter: 0; batch classifier loss: 0.697707
epoch 3; iter: 0; batch classifier loss: 0.668858
epoch 4; iter: 0; batch classifier loss: 0.683273
epoch 5; iter: 0; batch classifier loss: 0.661506
epoch 6; iter: 0; batch classifier loss: 0.672708
epoch 7; iter: 0; batch classifier loss: 0.666617
epoch 8; iter: 0; b

### Apply the plain model to test data

In [21]:
dataset_nodebiasing_train = plain_model.predict(diabetes_dataset_train)
dataset_nodebiasing_test = plain_model.predict(diabetes_dataset_test)

### Metrics for the dataset from plain model (without debiasing)

In [22]:
display(Markdown("#### Model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(diabetes_dataset_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.598303
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.589091


#### Model - without debiasing - classification metrics

Test set: Classification accuracy = 0.711039
Test set: Balanced classification accuracy = 0.610898
Test set: Disparate impact = 0.381679
Test set: Equal opportunity difference = -0.428571
Test set: Average odds difference = -0.538370
Test set: Theil_index = 0.075720


### Apply in-processing algorithm based on adversarial learning

In [23]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [24]:
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [25]:
debiased_model.fit(diabetes_dataset_train)

epoch 0; iter: 0; batch classifier loss: 0.719709; batch adversarial loss: 0.707410
epoch 1; iter: 0; batch classifier loss: 0.734156; batch adversarial loss: 0.704542
epoch 2; iter: 0; batch classifier loss: 0.726808; batch adversarial loss: 0.702275
epoch 3; iter: 0; batch classifier loss: 0.745609; batch adversarial loss: 0.701177
epoch 4; iter: 0; batch classifier loss: 0.731937; batch adversarial loss: 0.700088
epoch 5; iter: 0; batch classifier loss: 0.724922; batch adversarial loss: 0.698732
epoch 6; iter: 0; batch classifier loss: 0.713914; batch adversarial loss: 0.695718
epoch 7; iter: 0; batch classifier loss: 0.721002; batch adversarial loss: 0.695420
epoch 8; iter: 0; batch classifier loss: 0.706691; batch adversarial loss: 0.692693
epoch 9; iter: 0; batch classifier loss: 0.704806; batch adversarial loss: 0.690888
epoch 10; iter: 0; batch classifier loss: 0.713283; batch adversarial loss: 0.687029
epoch 11; iter: 0; batch classifier loss: 0.709272; batch adversarial loss:

### Apply the plain model to test data

In [26]:
dataset_debiasing_train = debiased_model.predict(diabetes_dataset_train)
dataset_debiasing_test = debiased_model.predict(diabetes_dataset_test)

In [27]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(diabetes_dataset_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.598303
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.589091


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.164958
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.019394


#### Model - without debiasing - classification metrics

Test set: Classification accuracy = 0.711039
Test set: Balanced classification accuracy = 0.610898
Test set: Disparate impact = 0.381679
Test set: Equal opportunity difference = -0.428571
Test set: Average odds difference = -0.538370
Test set: Theil_index = 0.075720


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.724026
Test set: Balanced classification accuracy = 0.697741
Test set: Disparate impact = 1.029963
Test set: Equal opportunity difference = 0.147151
Test set: Average odds difference = 0.120200
Test set: Theil_index = 0.186439


### We have observed how to use AI 360 fairness toolkit to eliminate the bias during preprocessing & inprocessing stages of model building & development. There's reduction from 42% to 14% with Equal opportunity difference & Average odds difference saw a reduction from 53% to 12% thereby eliminating the bias.